## Airline Delay Dataset

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

client = MongoClient("mongodb://localhost:27017")

# Airline delay
delay_df = pd.DataFrame(list(client["airline_delay_db"]["airline_delay_cause"].find()))
delay_df.drop(columns=['_id'], errors='ignore', inplace=True)

# Weather
weather_df = pd.DataFrame(list(client["flight_weather"]["noaa_weather_raw"].find()))
weather_df.drop(columns=['_id'], errors='ignore', inplace=True)

# Hurricanes
hurricane_df = pd.DataFrame(list(client["flight_weather_project"]["hurricane_ibtracs_na"].find()))
hurricane_df.drop(columns=['_id'], errors='ignore', inplace=True)


In [2]:
list(delay_df.columns)

['year',
 'month',
 'carrier',
 'carrier_name',
 'airport',
 'airport_name',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay']

In [3]:
import pandas as pd
import numpy as np

# you already have: delay_df = DataFrame from Mongo

# 1) make sure year & month are numeric
delay_df["year"] = pd.to_numeric(delay_df["year"], errors="coerce")
delay_df["month"] = pd.to_numeric(delay_df["month"], errors="coerce")

# drop rows where year or month is missing
delay_df = delay_df.dropna(subset=["year", "month"])

delay_df["year"] = delay_df["year"].astype(int)
delay_df["month"] = delay_df["month"].astype(int)

# 2) create a date for the first day of each month
delay_df["flight_month"] = pd.to_datetime(
    delay_df["year"].astype(str) + "-" +
    delay_df["month"].astype(str) + "-01",
    errors="coerce"
)

# 3) keep only your 15 airports
target_airports = [
    'ATL','DFW','DEN','ORD','LAX',
    'CLT','LAS','MCO','PHX','MIA',
    'SEA','EWR','JFK','SFO','BOS'
]

delay_df = delay_df[delay_df["airport"].isin(target_airports)]

In [4]:
delay_cols = [
    "arr_flights", "arr_del15",
    "carrier_ct", "weather_ct", "nas_ct", "security_ct", "late_aircraft_ct",
    "arr_cancelled", "arr_diverted",
    "arr_delay", "carrier_delay", "weather_delay", "nas_delay",
    "security_delay", "late_aircraft_delay"
]

for col in delay_cols:
    if col in delay_df.columns:
        delay_df[col] = pd.to_numeric(delay_df[col], errors="coerce")


In [5]:
delay_df = delay_df[(delay_df["year"] >= 2014) & (delay_df["year"] <= 2025)]

In [6]:
print(delay_df.shape)
delay_df.head()


(67803, 22)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,flight_month
4,2025,1,G4,Allegiant Air,EWR,"Newark, NJ: Newark Liberty International",31.0,5.0,2.17,0.00,...,0.00,1.0,0.0,446.0,336.0,0.0,110.0,0.0,0.0,2025-01-01
30,2025,1,G4,Allegiant Air,LAS,"Las Vegas, NV: Harry Reid International",575.0,151.0,27.17,16.06,...,52.18,5.0,0.0,20049.0,4556.0,5465.0,3328.0,15.0,6685.0,2025-01-01
31,2025,1,G4,Allegiant Air,LAX,"Los Angeles, CA: Los Angeles International",37.0,11.0,1.32,1.76,...,4.19,1.0,0.0,1191.0,43.0,209.0,157.0,0.0,782.0,2025-01-01
37,2025,1,G4,Allegiant Air,MCO,"Orlando, FL: Orlando International",32.0,3.0,0.00,0.17,...,0.00,1.0,0.0,64.0,0.0,4.0,60.0,0.0,0.0,2025-01-01
57,2025,1,G4,Allegiant Air,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",28.0,7.0,4.53,1.00,...,0.98,0.0,1.0,418.0,226.0,16.0,22.0,0.0,154.0,2025-01-01


In [7]:
!pip install psycopg2-binary

In [8]:
from sqlalchemy import create_engine

PG_USER = "postgres"          # or whatever user you use in pgAdmin
PG_PASSWORD = "123"  # <<< your REAL password, no quotes inside
PG_HOST = "localhost"
PG_PORT = "5432"
PG_DB   = "flight_project"

engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

delay_df.to_sql("stg_airline_delay", engine, if_exists="replace", index=False)


803

## NOAA Weather Dataset

In [9]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

client = MongoClient("mongodb://localhost:27017")
weather_cur = client["flight_weather"]["noaa_weather_raw"].find({})
weather_df = pd.DataFrame(list(weather_cur))
weather_df.drop(columns=['_id'], errors='ignore', inplace=True)

print(weather_df.shape)
weather_df.head()

(55296, 35)


,date,AWND,PGTM,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,...,WT04,PSUN,TSUN,WT05,airport,station_id,WT07,WT09,WESD,WT10
0,2025-01-01,8.3,1606.0,0.0,0.0,0.0,51.0,39.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-02,9.6,1508.0,0.0,0.0,0.0,43.0,33.0,280.0,230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-03,7.8,1439.0,0.0,0.0,0.0,39.0,31.0,230.0,230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-04,10.1,1302.0,0.0,0.0,0.0,33.0,28.0,270.0,320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-05,8.5,1614.0,0.0,0.0,0.0,33.0,28.0,310.0,310.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(list(weather_df.columns))

['date', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'ADPT', 'ASLP', 'ASTP', 'AWBT', 'RHAV', 'RHMN', 'RHMX', 'WT01', 'WT02', 'WT03', 'WT06', 'WT08', 'TAVG', 'WT04', 'PSUN', 'TSUN', 'WT05', 'airport', 'station_id', 'WT07', 'WT09', 'WESD', 'WT10']


In [11]:
import pandas as pd

# --- 1. Standardize date column ---
weather_df.rename(columns={"date": "obs_date"}, inplace=True)

# Convert obs_date to datetime
weather_df["obs_date"] = pd.to_datetime(weather_df["obs_date"], errors="coerce")

# --- 2. Standardize airport column ---
# You already have `airport`, so no need to rename 'station'
# (you also have a station_id column but airport is what we need)

# --- 3. Keep only your 15 airports ---
target_airports = [
    'ATL','DFW','DEN','ORD','LAX',
    'CLT','LAS','MCO','PHX','MIA',
    'SEA','EWR','JFK','SFO','BOS'
]

weather_df = weather_df[weather_df["airport"].isin(target_airports)]

# --- 4. Create year, month, and flight_month ---
weather_df["year"] = weather_df["obs_date"].dt.year
weather_df["month"] = weather_df["obs_date"].dt.month

weather_df["flight_month"] = pd.to_datetime(
    weather_df["year"].astype(str) + "-" +
    weather_df["month"].astype(str) + "-01"
)

weather_df.head()
print(weather_df.shape)


(54601, 38)


In [12]:
# Select numeric weather columns
numeric_cols = [
    "AWND",   # average wind speed
    "PRCP",   # precipitation
    "SNOW",   # snowfall
    "TMAX",   # max temp
    "TMIN",   # min temp
    "TAVG",   # average temp
    "WSF2",   # max 2-min wind speed
    "WSF5",   # max 5-sec wind speed
    "AWBT",   # average wet bulb temp
    "RHAV",   # avg relative humidity
]

numeric_cols = [c for c in numeric_cols if c in weather_df.columns]

weather_monthly = weather_df.groupby(
    ["airport", "flight_month"], as_index=False
)[numeric_cols].mean()

weather_monthly.head()


,airport,flight_month,AWND,PRCP,SNOW,TMAX,TMIN,TAVG,WSF2,WSF5,AWBT,RHAV
0,ATL,2015-01-01,8.696774,0.140645,0.0,52.516129,33.774194,42.903226,19.403226,25.035484,31.193548,61.741935
1,ATL,2015-02-01,9.551724,0.164138,0.0,50.241379,31.068966,40.517241,20.627586,27.293103,NaN,NaN
2,ATL,2015-03-01,8.162500,0.097187,0.0,66.906250,47.218750,56.218750,17.478125,22.115625,100.843750,67.187500
3,ATL,2015-04-01,8.461290,0.270968,0.0,74.838710,56.677419,65.483871,18.354839,24.196774,146.032258,65.483871
4,ATL,2015-05-01,6.328125,0.138750,0.0,82.375000,62.906250,72.781250,17.153125,22.809375,175.343750,61.218750


In [13]:
from sqlalchemy import create_engine

PG_USER = "postgres"          # or whatever user you use in pgAdmin
PG_PASSWORD = "123"  # <<< your REAL password, no quotes inside
PG_HOST = "localhost"
PG_PORT = "5432"
PG_DB   = "flight_project"

engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

weather_monthly.to_sql(
    "stg_weather",
    engine,
    if_exists="replace",
    index=False
)

814

## Hurricane Dataset

In [30]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

# ----- Mongo connection (same as before) -----
client = MongoClient("mongodb://localhost:27017")

hur_cur = client["flight_weather_project"]["hurricane_ibtracs_na"].find({})
hurricane_df = pd.DataFrame(list(hur_cur))

# drop Mongo _id
hurricane_df.drop(columns=["_id"], errors="ignore", inplace=True)

hurricane_df.head()


,SID,SEASON,BASIN,SUBBASIN,NAME,ISO_TIME,USA_LAT,USA_LON,USA_WIND,USA_PRES,USA_SSHS
0,2015126N27281,2015.0,None,None,ANA,2015-05-06 06:00:00,26.8,-79.2,25,1016,-3
1,2015126N27281,2015.0,None,None,ANA,2015-05-06 09:00:00,27.5,-78.9,25,1016,-3
2,2015126N27281,2015.0,None,None,ANA,2015-05-06 12:00:00,28.2,-78.5,25,1015,-3
3,2015126N27281,2015.0,None,None,ANA,2015-05-06 15:00:00,29.0,-78.1,25,1015,-3
4,2015126N27281,2015.0,None,None,ANA,2015-05-06 18:00:00,29.7,-77.8,25,1014,-3


In [32]:
print(list(hurricane_df.columns))

['SID', 'SEASON', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME', 'USA_LAT', 'USA_LON', 'USA_WIND', 'USA_PRES', 'USA_SSHS']


In [33]:
import pandas as pd
import numpy as np

# Rename to cleaner names
hurricane_df = hurricane_df.rename(columns={
    "SID": "sid",
    "SEASON": "season",
    "BASIN": "basin",
    "SUBBASIN": "subbasin",
    "NAME": "name",
    "ISO_TIME": "iso_time",
    "USA_LAT": "lat",
    "USA_LON": "lon",
    "USA_WIND": "wind_kt",
    "USA_PRES": "pressure_hpa",
    "USA_SSHS": "sshs_category"
})

In [34]:
# Convert columns to correct dtypes
hurricane_df["iso_time"] = pd.to_datetime(hurricane_df["iso_time"], errors="coerce")
hurricane_df["season"] = pd.to_numeric(hurricane_df["season"], errors="coerce")
hurricane_df["lat"] = pd.to_numeric(hurricane_df["lat"], errors="coerce")
hurricane_df["lon"] = pd.to_numeric(hurricane_df["lon"], errors="coerce")
hurricane_df["wind_kt"] = pd.to_numeric(hurricane_df["wind_kt"], errors="coerce")
hurricane_df["pressure_hpa"] = pd.to_numeric(hurricane_df["pressure_hpa"], errors="coerce")
hurricane_df["sshs_category"] = pd.to_numeric(hurricane_df["sshs_category"], errors="coerce")


In [35]:
hurricane_df = hurricane_df[hurricane_df["season"] >= 2015]
hurricane_df = hurricane_df.dropna(subset=["iso_time"])


In [36]:
hurricane_df["flight_month"] = hurricane_df["iso_time"].dt.to_period("M").dt.to_timestamp()


In [23]:
hurricane_df.head()


,sid,season,basin,subbasin,name,iso_time,lat,lon,wind_kt,pressure_hpa,sshs_category,flight_month
0,2015126N27281,2015.0,None,None,ANA,2015-05-06 06:00:00,26.8,-79.2,25,1016.0,-3,2015-05-01
1,2015126N27281,2015.0,None,None,ANA,2015-05-06 09:00:00,27.5,-78.9,25,1016.0,-3,2015-05-01
2,2015126N27281,2015.0,None,None,ANA,2015-05-06 12:00:00,28.2,-78.5,25,1015.0,-3,2015-05-01
3,2015126N27281,2015.0,None,None,ANA,2015-05-06 15:00:00,29.0,-78.1,25,1015.0,-3,2015-05-01
4,2015126N27281,2015.0,None,None,ANA,2015-05-06 18:00:00,29.7,-77.8,25,1014.0,-3,2015-05-01


In [37]:
import pandas as pd
import numpy as np


# ====== MONTHLY AGGREGATION ======
# group by month and summarize all columns
hurricane_monthly_df = (
    hurricane_df
    .groupby("flight_month", as_index=False)
    .agg({
        # how many unique storms in that month
        "sid": "nunique",

        # these are mostly identifiers, so just take the first value
        "season": "first",
        "basin": "first",
        "subbasin": "first",

        # if multiple storm names in one month, you can also join them later if you want
        "name": "first",

        # numeric columns – take averages (you can change to max/min if you prefer)
        "lat": "mean",
        "lon": "mean",
        "wind_kt": "mean",
        "pressure_hpa": "mean",

        # max sshs_category in that month (strongest category)
        "sshs_category": "max",
    })
)

# rename sid -> n_storms so it’s clear
hurricane_monthly_df = hurricane_monthly_df.rename(columns={"sid": "n_storms"})

hurricane_monthly_df.head()


,flight_month,n_storms,season,basin,subbasin,name,lat,lon,wind_kt,pressure_hpa,sshs_category
0,2015-05-01,1,2015.0,None,None,ANA,33.411111,-77.037037,35.777778,1006.351852,0
1,2015-06-01,1,2015.0,None,GM,BILL,33.588095,-93.378571,26.500000,1001.928571,0
2,2015-07-01,1,2015.0,None,None,CLAUDETTE,38.737931,-66.662069,34.310345,1006.275862,0
3,2015-08-01,3,2015.0,None,CS,DANNY,14.006481,-45.023148,51.101852,999.009259,3
4,2015-09-01,6,2015.0,None,None,FRED,19.882182,-45.375636,34.170909,1004.923636,2


In [39]:
from sqlalchemy import create_engine

PG_USER = "postgres"          # or whatever user you use in pgAdmin
PG_PASSWORD = "123"  # <<< your REAL password, no quotes inside
PG_HOST = "localhost"
PG_PORT = "5432"
PG_DB   = "flight_project"

engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

hurricane_monthly_df.to_sql(
    "stg_hurricane",
    engine,
    if_exists="replace",
    index=False
)

72